<a href="https://colab.research.google.com/github/newmantic/Binomial_tree/blob/main/Binomial_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def binomial_tree_european(S0, K, T, r, sigma, N, option_type='call'):
    """
    Binomial Tree model for pricing European options.

    Parameters:
    - S0: Current stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free interest rate (annual)
    - sigma: Volatility of the underlying asset (annual)
    - N: Number of steps in the binomial tree
    - option_type: 'call' for call option, 'put' for put option

    Returns:
    - option_price: The price of the European option
    """
    # Calculate parameters
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(N + 1)
    for i in range(N + 1):
        ST[i] = S0 * (u ** i) * (d ** (N - i))

    # Initialize option values at maturity
    if option_type == 'call':
        option_values = np.maximum(0, ST - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - ST)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    # Step back through the tree
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])

    return option_values[0]

In [2]:
# Parameters
S0 = 100         # Current stock price
K = 100          # Strike price
T = 1            # Time to maturity (1 year)
r = 0.05         # Risk-free interest rate (5% per annum)
sigma = 0.2      # Volatility (20% per annum)
N = 3            # Number of steps in the binomial tree

# Price the European call option
call_price = binomial_tree_european(S0, K, T, r, sigma, N, option_type='call')
print(f"European Call Option Price: {call_price:.2f}")

# Price the European put option
put_price = binomial_tree_european(S0, K, T, r, sigma, N, option_type='put')
print(f"European Put Option Price: {put_price:.2f}")

European Call Option Price: 11.04
European Put Option Price: 6.17


In [3]:
def binomial_tree_american(S0, K, T, r, sigma, N, option_type='call'):
    """
    Binomial Tree model for pricing American options.

    Parameters:
    - S0: Current stock price
    - K: Strike price
    - T: Time to maturity (in years)
    - r: Risk-free interest rate (annual)
    - sigma: Volatility of the underlying asset (annual)
    - N: Number of steps in the binomial tree
    - option_type: 'call' for call option, 'put' for put option

    Returns:
    - option_price: The price of the American option
    """
    # Calculate parameters
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Initialize asset prices at maturity
    ST = np.zeros(N + 1)
    for i in range(N + 1):
        ST[i] = S0 * (u ** i) * (d ** (N - i))

    # Initialize option values at maturity
    if option_type == 'call':
        option_values = np.maximum(0, ST - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - ST)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")

    # Step back through the tree, checking for early exercise
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            ST[i] = S0 * (u ** i) * (d ** (j - i))
            hold_value = np.exp(-r * dt) * (p * option_values[i + 1] + (1 - p) * option_values[i])
            exercise_value = max(ST[i] - K, 0) if option_type == 'call' else max(K - ST[i], 0)
            option_values[i] = max(hold_value, exercise_value)

    return option_values[0]

In [4]:
# Price the American call option
american_call_price = binomial_tree_american(S0, K, T, r, sigma, N, option_type='call')
print(f"American Call Option Price: {american_call_price:.2f}")

# Price the American put option
american_put_price = binomial_tree_american(S0, K, T, r, sigma, N, option_type='put')
print(f"American Put Option Price: {american_put_price:.2f}")

American Call Option Price: 11.04
American Put Option Price: 6.50
